In [1]:
import os

import gmsh
import matplotlib.pyplot as plt
import meshio
import numpy as np
import pandas as pd

import commons, geometry, utils

In [2]:
Lsep = 25
Lcat = 50
LZ = int(Lsep + Lcat)
R = 235
Rp = 6
eps_se = 0.5
eps_am = 1 - eps_se
markers = commons.Markers()
phases = commons.Phases()
CELL_TYPES = commons.CellTypes()
start_zpos = 50
scale = (800/470) * 1e-6
scale_factor = [scale, scale, scale]
dimensions = f'470-470-{LZ}'
name_of_study = 'reaction_distribution'
outdir = f"output/{name_of_study}/{dimensions}/{eps_am}/"
utils.make_dir_if_missing(outdir)
msh_path = os.path.join(outdir, 'laminate.msh')
tetr_path = os.path.join(outdir, 'tetr.xdmf')
tria_path = os.path.join(outdir, 'tria.xdmf')

## Build Laminate Geometry of Cylindrical SE Strands

In [3]:
df = 470 * pd.read_csv('data/laminate.csv')

In [7]:
gmsh.initialize()
gmsh.model.add('laminate')
gmsh.option.setNumber('General.Verbosity', 1)
box_se = gmsh.model.occ.addCylinder(R, R, 0, 0, 0, Lsep, 235)
gmsh.model.occ.synchronize()
box_am = gmsh.model.occ.addCylinder(R, R, Lsep, 0, 0, Lcat, 235)
gmsh.model.occ.synchronize()
cylinders = []
spheres = []

for idx in range(df.shape[0]):
    x, y, _ = df.loc[idx, :]
    if ((235 - abs(x)) ** 2 + (235 - abs(y)) ** 2) ** 0.5 >= 235:
        continue
    cyl = gmsh.model.occ.addCylinder(x, y, Lsep, 0, 0, Lcat - Rp, Rp)
    gmsh.model.occ.synchronize()
    cylinders.append(cyl)
    gmsh.model.occ.synchronize()

se_phase = gmsh.model.occ.fuse([(3, box_se)], [(3, c) for c in cylinders], removeTool=False)
gmsh.model.occ.synchronize()
union = gmsh.model.occ.cut([(3, box_am)], [(3, c) for c in cylinders])
gmsh.model.occ.synchronize()

vols = gmsh.model.occ.getEntities(3)
se_vol = gmsh.model.addPhysicalGroup(3, [vols[0][1]], markers.electrolyte)
gmsh.model.occ.synchronize()
gmsh.model.setPhysicalName(3, se_vol, "electrolyte")
gmsh.model.occ.synchronize()
am_vol = gmsh.model.addPhysicalGroup(3, [vols[1][1]], markers.positive_am)
gmsh.model.occ.synchronize()
gmsh.model.setPhysicalName(3, am_vol, "positive_am")
gmsh.model.occ.synchronize()
right = []
left = []
insulated = []
insulated_se = []
insulated_am = []
interface = []
for surf in gmsh.model.occ.getEntities(2):
    com = gmsh.model.occ.getCenterOfMass(surf[0], surf[1])
    if np.isclose(com[2], Lsep + Lcat):
        right.append(surf[1])
    elif np.isclose(com[2], 0):
        left.append(surf[1])
    elif np.isclose(com[2], 0.5 * Lsep):
        if np.isclose(com[0], 235) or np.isclose(com[1], 235):
            insulated_se.append(surf[1])
        else:
            interface.append(surf[1])
    elif np.isclose(com[2], Lsep + 0.5 * (Lcat - Rp)):
        if np.isclose(com[0], 235) or np.isclose(com[1], 235):
            print("here")
            insulated_am.append(surf[1])
        else:
            interface.append(surf[1])
    else:
        print(com)
        interface.append(surf[1])

left = gmsh.model.addPhysicalGroup(2, left, markers.left)
gmsh.model.setPhysicalName(2, left, "left")
right = gmsh.model.addPhysicalGroup(2, right, markers.right)
gmsh.model.setPhysicalName(2, right, "right")
insulated_se = gmsh.model.addPhysicalGroup(2, insulated_se, markers.insulated_electrolyte)
gmsh.model.setPhysicalName(2, insulated_se, "insulated_electrolyte")
insulated_am = gmsh.model.addPhysicalGroup(2, insulated_am, markers.insulated_positive_am)
gmsh.model.setPhysicalName(2, insulated_am, "insulated_am")
insulated = gmsh.model.addPhysicalGroup(2, insulated, markers.insulated)
gmsh.model.setPhysicalName(2, insulated, "insulated")
electrolyte_v_positive_am = gmsh.model.addPhysicalGroup(2, interface, markers.electrolyte_v_positive_am)
gmsh.model.setPhysicalName(2, electrolyte_v_positive_am, "electrolyte_positive_am_interface")
gmsh.model.occ.synchronize()
gmsh.model.mesh.generate(3)
gmsh.write(msh_path)
gmsh.finalize()

(235.04749217795484, 234.56446428352433, 25.0)
(324.4324920821075, 453.33628536400306, 25.0)
(234.48863486078474, 233.75157716886156, 50.0172236652112)
(235.04749217795484, 234.56446428352433, 25.0)
(322.40613526205715, 448.38923300516075, 69.0)
(45.804107517042, 124.14119895998684, 69.0)
(79.78982091474148, 105.2536410543392, 69.0)
(168.79431589399096, 23.285014491063848, 69.0)
(173.31925591406647, 66.94579935440389, 69.0)
(213.71458400581824, 54.14123547619178, 69.0)
(130.2911693184667, 74.13308367070672, 69.0)
(130.44447152420543, 111.79337665021927, 69.0)
(223.37165170684509, 101.88837550721556, 69.0)
(105.13029496809658, 139.4998754239964, 69.0)
(106.68909580023366, 180.0757156204236, 69.0)
(18.229913665311695, 207.42521248345125, 69.0)
(61.37305386225785, 242.92100872747028, 69.0)
(140.2693073149808, 155.89270339935962, 69.0)
(221.21234013712103, 141.9908080768383, 69.0)
(189.7717855784309, 162.81523311033962, 69.0)
(162.81483352111533, 189.0770154156772, 69.0)
(130.9748088775769

In [5]:
msh = meshio.read(msh_path)
tetr_unscaled = geometry.create_mesh(msh, CELL_TYPES.tetra)
tetr_unscaled.write(tetr_path)
tetr_scaled = geometry.scale_mesh(tetr_unscaled, CELL_TYPES.tetra, scale_factor=scale_factor)
tetr_scaled.write(tetr_path)
tria_unscaled = geometry.create_mesh(msh, CELL_TYPES.triangle)
tria_unscaled.write(tria_path)
tria_scaled = geometry.scale_mesh(tria_unscaled, CELL_TYPES.triangle, scale_factor=scale_factor)
tria_scaled.write(tria_path)